# PUBG prediction 

https://www.kaggle.com/c/pubg-finish-placement-prediction/data

https://www.kaggle.com/anycode/simple-nn-baseline-4

## Install Kaggle api 
https://github.com/Kaggle/kaggle-api

In [16]:
#! sudo /opt/anaconda3/bin/pip install kaggle --upgrade
#! mkdir -p ~/.kaggle/
# ! mv ~/kaggle.json ~/.kaggle/

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai import *
from fastai.tabular import * 
from tqdm import tqdm

In [3]:
path = Path(Config.get_key('data_path')).expanduser()/'pubg'
os.makedirs(path, exist_ok=True)
path

PosixPath('/home/jupyter/.fastai/data/pubg')

In [4]:
# ! /opt/anaconda3/bin/kaggle competitions download -c pubg-finish-placement-prediction -f train_V2.csv -p {path}
# ! /opt/anaconda3/bin/kaggle competitions download -c pubg-finish-placement-prediction -f test_V2.csv -p {path}
# ! unzip {path}/train_V2.csv.zip -d {path}
# ! unzip {path}/test_V2.csv.zip -d {path}
# !chmod 775 -R {path}

In [5]:
df = pd.read_csv((path/'train_V2.csv').as_posix())

In [47]:
df.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [48]:
# df = df[df['maxPlace'] > 1]
# df['totalDistance'] = df['rideDistance'] + df["walkDistance"] + df["swimDistance"]
# print("remove some columns")
# target = 'winPlacePerc'
# features = list(df.columns)
# features.remove("Id")
# features.remove("matchId")
# features.remove("groupId")

# features.remove("matchType")
# y = None
# print("get target")
# y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
# features.remove(target)
# agg = df.groupby(['matchId','groupId'])[features].agg('mean')
# agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
# df_out = agg.reset_index()[['matchId','groupId']]
# df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
# df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
# print("get group max feature")
# agg = df.groupby(['matchId','groupId'])[features].agg('max')
# agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
# df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
# df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])

# print("get group min feature")
# agg = df.groupby(['matchId','groupId'])[features].agg('min')
# agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
# df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
# df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])

# print("get group size feature")
# agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
# df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])

# print("get match mean feature")
# agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
# df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
# print("get match size feature")
# agg = df.groupby(['matchId']).size().reset_index(name='match_size')
# df_out = df_out.merge(agg, how='left', on=['matchId'])

# df_out.drop(["matchId", "groupId"], axis=1, inplace=True)

# X = np.array(df_out, dtype=np.float64)

# feature_names = list(df_out.columns)


In [49]:
# df_out.head()

In [9]:
def groupId_func(row):
    return groupId_map[row.groupId]

def matchId_func(row):
    return matchId_map[row.matchId]

def matchType_func(row):
    return matchType_map[row.matchType]
    
groupId_map = {v:i for i,v in enumerate(df['groupId'].unique().tolist())} 
matchType_map = {v:i for i,v in enumerate(df['matchType'].unique().tolist())} 
matchId_map = {v:i for i,v in enumerate(df['matchId'].unique().tolist())} 

df['groupIdN'] = df.apply(lambda row: groupId_func(row),axis=1)
df['matchTypeN'] = df.apply(lambda row: matchType_func(row),axis=1)
df['matchIdN'] = df.apply(lambda row: matchId_func(row),axis=1)

df = df.drop(['Id', 'groupId', 'matchType', 'matchId'], axis=1) 

In [10]:

cat_names = ['groupIdN', 'matchIdN', 'assists', 'boosts', 'DBNOs', 'headshotKills', 
             'heals', 'killPlace', 'killPoints', 'kills', 'killStreaks', 
             'matchDuration', 'matchTypeN', 'maxPlace', 'numGroups', 
             'rankPoints', 'revives', 'roadKills', 'teamKills', 'vehicleDestroys', 
             'weaponsAcquired', 'winPoints']
    
dep_var = 'winPlacePerc'

In [18]:
ldf = len(df)
ldf80 = int(ldf*.8)

In [19]:
train_df, valid_df = df[:ldf80].copy(),df[ldf80:].copy()
# train_df.head()
tfms = [FillMissing, Categorify]
data = TabularDataBunch.from_df(path, train_df, valid_df, dep_var=dep_var, tfms=tfms, cat_names=cat_names)

In [ ]:
learner = get_tabular_learner(data, layers=[200,100], metrics=accuracy)

In [ ]:
learner.fit(1, lr=1e-3)

epoch,train_loss,valid_loss,accuracy


In [6]:
!jupyter <app> --existing kernel-42c1d731-08ba-4626-b041-018f357e7823.json

/bin/sh: 1: cannot open app: No such file


In [5]:
!jupyter <app> --existing

/bin/sh: 1: cannot open app: No such file


In [1]:
%connect_info  

{
  "shell_port": 37313,
  "iopub_port": 33805,
  "stdin_port": 39485,
  "control_port": 57611,
  "hb_port": 54189,
  "ip": "127.0.0.1",
  "key": "0aabf728-c112e8e05913c75a169b0cbf",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-69deba78-fe81-4e01-8950-0ac41b328553.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [5]:
!jupyter notebook --existing kernel-17a843f6-b5a4-4f0d-bdfc-b66c082bfe3b.json --browser

usage: jupyter-notebook [-h] [--log-level NOTEBOOKAPP.LOG_LEVEL]
                        [--config NOTEBOOKAPP.CONFIG_FILE]
                        [--ip NOTEBOOKAPP.IP] [--port NOTEBOOKAPP.PORT]
                        [--port-retries NOTEBOOKAPP.PORT_RETRIES]
                        [--transport KERNELMANAGER.TRANSPORT]
                        [--keyfile NOTEBOOKAPP.KEYFILE]
                        [--certfile NOTEBOOKAPP.CERTFILE]
                        [--client-ca NOTEBOOKAPP.CLIENT_CA]
                        [--notebook-dir NOTEBOOKAPP.NOTEBOOK_DIR]
                        [--browser NOTEBOOKAPP.BROWSER]
                        [--pylab [NOTEBOOKAPP.PYLAB]] [--debug]
                        [--generate-config] [-y] [--no-browser] [--no-mathjax]
                        [--allow-root] [--script] [--no-script]
jupyter-notebook: error: argument --browser: expected one argument


In [3]:
! jupyter-notebook cmd list

[C 17:04:19.616 NotebookApp] No such file or directory: /home/jupyter/my_notes/cmd
